In [1]:
import pandas as pd
import numpy as np
import warnings
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense,Flatten
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from keras.layers import Dropout,Conv1D, Conv2D,MaxPooling2D,MaxPooling1D,UpSampling1D,Reshape,Activation,BatchNormalization,LeakyReLU
from keras.models import Sequential
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
tsa = pd.read_csv("./tsa.csv",header=None)

In [3]:

tsa.columns = ["time"]

In [4]:
pat = pd.read_csv("./moss_plos_one_data.csv")

In [5]:
new_pat = pd.concat([pat,tsa],axis=1)

In [6]:
b = new_pat[new_pat.y==True]
id_true = list(set(list(b.id)))

In [7]:
# true_patient = new_pat[new_pat.id.isin(id_true)]
# true_patient["num"] = 0
# true_patient.shape
p = new_pat[(new_pat["tte"]<=0)&(new_pat["tte"]>-0.5)]

In [8]:
true_patient = p[p["tte"].notnull()]

In [9]:
true_patient = true_patient[true_patient.id.isin(id_true)]

In [10]:
true_patient.head()

,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,y,...,s2.edrk,s8.edrk,s24.edrk,srr,dfa,cosen,lds,af,AF,time
26318,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.000244,-0.000119,-0.000012,0.026143,1.180124,-2.378111,1.004765,0.0,0.0,6187.0
26319,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.000206,-0.000106,-0.000010,0.037133,0.897787,-2.352599,0.901798,0.0,0.0,6202.0
26320,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.000150,-0.000094,-0.000009,0.044280,0.909218,-2.186083,1.141100,0.0,0.0,6217.0
26321,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.000114,-0.000083,-0.000008,0.038783,0.728632,-2.361748,1.011130,0.0,0.0,6232.0
26322,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.000172,-0.000068,-0.000007,0.025394,0.877550,-2.299310,1.553938,0.0,0.0,6247.0


In [11]:
true_patient["num"] = 1
true_patient["cond"] = 0
true_patient.cond[true_patient.y==True] =1

In [12]:
true_patient.head()

,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,y,...,s24.edrk,srr,dfa,cosen,lds,af,AF,time,num,cond
26318,133,63.7,wht,Other,14,5,0,1,8.0,True,...,-0.000012,0.026143,1.180124,-2.378111,1.004765,0.0,0.0,6187.0,1,1
26319,133,63.7,wht,Other,14,5,0,1,8.0,True,...,-0.000010,0.037133,0.897787,-2.352599,0.901798,0.0,0.0,6202.0,1,1
26320,133,63.7,wht,Other,14,5,0,1,8.0,True,...,-0.000009,0.044280,0.909218,-2.186083,1.141100,0.0,0.0,6217.0,1,1
26321,133,63.7,wht,Other,14,5,0,1,8.0,True,...,-0.000008,0.038783,0.728632,-2.361748,1.011130,0.0,0.0,6232.0,1,1
26322,133,63.7,wht,Other,14,5,0,1,8.0,True,...,-0.000007,0.025394,0.877550,-2.299310,1.553938,0.0,0.0,6247.0,1,1


In [13]:
true_patient["cum_sum"] = true_patient.groupby("id")["cond"].cumsum()

In [14]:
t_p = true_patient[(true_patient.cum_sum!=0)]

In [15]:
t_p["new_count"] = 1

In [16]:
filterPat = t_p.groupby("id")["new_count"].sum().to_frame().reset_index()
filterPat.head(10)

,id,new_count
0,133,40
1,187,40
2,191,4
3,206,40
4,282,40
5,295,23
6,307,80
7,348,39
8,414,6
9,427,40


In [17]:
t_p[t_p.id==133]

,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,y,...,dfa,cosen,lds,af,AF,time,num,cond,cum_sum,new_count
26318,133,63.7,wht,Other,14,5,0,1,8.0,True,...,1.180124,-2.378111,1.004765,0.0,0.0,6187.0,1,1,1,1
26319,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.897787,-2.352599,0.901798,0.0,0.0,6202.0,1,1,2,1
26320,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.909218,-2.186083,1.141100,0.0,0.0,6217.0,1,1,3,1
26321,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.728632,-2.361748,1.011130,0.0,0.0,6232.0,1,1,4,1
26322,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.877550,-2.299310,1.553938,0.0,0.0,6247.0,1,1,5,1
26323,133,63.7,wht,Other,14,5,0,1,8.0,True,...,1.272835,-2.252002,1.254278,0.0,0.0,6262.0,1,1,6,1
26324,133,63.7,wht,Other,14,5,0,1,8.0,True,...,1.345833,-2.319829,0.844162,0.0,0.0,6277.0,1,1,7,1
26325,133,63.7,wht,Other,14,5,0,1,8.0,True,...,1.270736,-2.281478,0.944826,0.0,0.0,6292.0,1,1,8,1
26326,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.712641,-2.249134,1.274229,0.0,0.0,6307.0,1,1,9,1
26327,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.781597,-2.361413,1.525636,0.0,0.0,6322.0,1,1,10,1


In [18]:
filtered_id =  list(filterPat.id[filterPat.new_count>=4*6])
new_df = t_p[t_p.id.isin(filtered_id)]

In [19]:
len(filtered_id)

251

In [20]:
# new_df2 = new_df.groupby("id").head(96)


In [21]:
# reverse_df = new_df2[::-1]
reverse_df = new_df[::-1]

In [22]:
# new_df2 = new_df.loc[new_df.y==True]

In [23]:
reverse_df.head()

,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,y,...,dfa,cosen,lds,af,AF,time,num,cond,cum_sum,new_count
2211188,8060,67.92,wht,Cardiology,26,8,0,1,619.0,True,...,0.631044,-2.231604,1.618099,0.0,0.0,7889.0,1,1,40,1
2211187,8060,67.92,wht,Cardiology,26,8,0,1,619.0,True,...,0.631044,-2.231604,1.618099,0.0,0.0,7874.0,1,1,39,1
2211186,8060,67.92,wht,Cardiology,26,8,0,1,619.0,True,...,0.631044,-2.231604,1.618099,0.0,0.0,7859.0,1,1,38,1
2211185,8060,67.92,wht,Cardiology,26,8,0,1,619.0,True,...,0.631044,-2.231604,1.618099,0.0,0.0,7844.0,1,1,37,1
2211184,8060,67.92,wht,Cardiology,26,8,0,1,619.0,True,...,0.631044,-2.231604,1.618099,0.0,0.0,7829.0,1,1,36,1


In [24]:
# reverse_df = new_df2[::-1]
# reverse_df_24 = new_df2.groupby("id").head(48)
reverse_df_24 = reverse_df.groupby("id").head(24)

In [25]:
reverse_df_24.shape

(6024, 67)

In [26]:
# reverse_df_12 = reverse_df_24.groupby("id").head(48)
reverse_df_24 = reverse_df_24[::-1]

In [27]:
reverse_df_24.head()

,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,y,...,dfa,cosen,lds,af,AF,time,num,cond,cum_sum,new_count
26334,133,63.7,wht,Other,14,5,0,1,8.0,True,...,1.075764,-2.262676,0.864124,0.0,0.0,6427.0,1,1,17,1
26335,133,63.7,wht,Other,14,5,0,1,8.0,True,...,1.071926,-2.331087,0.781297,0.0,0.0,6442.0,1,1,18,1
26336,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.928452,-2.275927,0.702950,0.0,0.0,6457.0,1,1,19,1
26337,133,63.7,wht,Other,14,5,0,1,8.0,True,...,0.775527,-2.342968,0.920636,0.0,0.0,6472.0,1,1,20,1
26338,133,63.7,wht,Other,14,5,0,1,8.0,True,...,1.002887,-2.261134,1.026540,0.0,0.0,6487.0,1,1,21,1


In [28]:
reverse_df_24["t2"] = reverse_df_24.time.shift(1)

In [29]:
reverse_df_24["diff"] = reverse_df_24.time - reverse_df_24["t2"]

In [30]:
reverse_df_24["diff"].fillna(15,inplace=True)


In [31]:
first_row = reverse_df_24.groupby("id").apply(lambda x: x.iloc[0:1]).reset_index(level=1).reset_index(drop=True)

In [32]:
first_row.rename(columns = {"level_1":"ind"},inplace=True)

In [33]:
first_row.head()

,ind,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,...,lds,af,AF,time,num,cond,cum_sum,new_count,t2,diff
0,26334,133,63.70,wht,Other,14,5,0,1,8.0,...,0.864124,0.0,0.0,6427.0,1,1,17,1,NaN,15.0
1,35966,187,52.23,wht,Cardiology,22,11,0,1,9.0,...,1.618099,0.0,0.0,6710.0,1,1,17,1,6772.0,-62.0
2,40649,206,55.34,blck,Other Medicine,9,2,0,1,11.0,...,1.808414,0.0,0.0,7021.0,1,1,17,1,7055.0,-34.0
3,55956,282,67.28,wht,Cardiothoracic Surgery,14,4,0,1,15.0,...,1.731868,0.0,0.0,6810.0,1,1,17,1,7366.0,-556.0
4,65619,307,47.69,wht,Cardiology,73,24,1,2,19.0,...,1.014304,0.0,0.0,99414.0,1,1,57,1,7155.0,92259.0


In [34]:
c1 = reverse_df_24.groupby("id").apply(lambda x: x.iloc[1:]).reset_index(level=1).reset_index(drop=True)

In [35]:
# c1.groupby("id")["diff"].max()
c1.rename(columns = {"level_1":"ind"},inplace=True)

In [36]:
max_diff = c1.groupby("id")["diff"].max().to_frame().reset_index()
lt_60_id  = max_diff[max_diff["diff"] <= 45].id

In [37]:
lt_60_id.shape #227 paitents left

(227,)

In [38]:
first_row_left  = first_row[first_row.id.isin(lt_60_id.tolist())]
c1_left = c1[c1.id.isin(lt_60_id.tolist())]

In [39]:
empty_df = pd.DataFrame()
i = 0 
for index, row in c1_left.iterrows():
    num = row["diff"]/15.0
    if num >1:
        empty_df= empty_df.append([c1_left.iloc[i-1,]]*int(num)) 
    i+=1


            
         
        

In [40]:
c1_left.shape

(5221, 70)

In [41]:
c_name = c1_left.columns

In [42]:
empty_df = empty_df[c_name]

In [43]:
c1_left.shape

(5221, 70)

In [44]:
merged_df1 = pd.concat([c1_left,first_row_left,empty_df],axis=0)

In [45]:
#sort the df by index col
merged_df1.sort_values("ind",inplace=True) #5515-227-5221 = 67 rows added

In [46]:
first_row_left.shape

(227, 70)

In [47]:
merged_df1.shape

(5515, 70)

In [48]:
merged_df2 = merged_df1.groupby("id").head(24)

In [49]:
df_12 = merged_df2.copy()
df_12.head()

,ind,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,...,lds,af,AF,time,num,cond,cum_sum,new_count,t2,diff
0,26334,133,63.7,wht,Other,14,5,0,1,8.0,...,0.864124,0.0,0.0,6427.0,1,1,17,1,NaN,15.0
0,26335,133,63.7,wht,Other,14,5,0,1,8.0,...,0.781297,0.0,0.0,6442.0,1,1,18,1,6427.0,15.0
1,26336,133,63.7,wht,Other,14,5,0,1,8.0,...,0.702950,0.0,0.0,6457.0,1,1,19,1,6442.0,15.0
2,26337,133,63.7,wht,Other,14,5,0,1,8.0,...,0.920636,0.0,0.0,6472.0,1,1,20,1,6457.0,15.0
3,26338,133,63.7,wht,Other,14,5,0,1,8.0,...,1.026540,0.0,0.0,6487.0,1,1,21,1,6472.0,15.0


In [50]:
id_list = list(set(pat['id']))  # 8105
id_false = [id for id in id_list if id not in id_true]
false_patient = pat[pat.id.isin(id_false)]

In [51]:
false_patient["cond"] = 1
false_count = false_patient.groupby("id")["cond"].sum().to_frame()
false_count = false_count[false_count.cond>24]

In [52]:
false_id = false_count.index

In [53]:
false_patient = pat[pat.id.isin(false_id)]
false_patient.head()

,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,y,...,edrk,s2.edrk,s8.edrk,s24.edrk,srr,dfa,cosen,lds,af,AF
0,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,23.527702,0.000002,0.000001,-4.918909e-07,0.027926,0.722903,-2.391665,1.670820,0.0,0.0
1,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,25.659513,0.002369,0.002369,2.368679e-03,0.031948,0.728738,-2.432732,1.995264,0.0,0.0
2,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,26.504228,0.001654,0.001654,1.653626e-03,0.023217,0.804216,-2.459413,2.058334,0.0,0.0
3,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,26.203203,0.000986,0.000986,9.856909e-04,0.010812,0.974394,-2.455503,1.986356,0.0,0.0
4,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,24.115443,0.000191,0.000191,1.910191e-04,0.007782,0.768893,-2.429776,1.900642,0.0,0.0


In [54]:
#randomly pick 12 hours 
#split training and test set
false_id = list(false_id)
true_id = list(set(df_12.id))
np.random.seed(200)
np.random.shuffle(list(false_id))
np.random.shuffle(list(true_id))


In [55]:
false_patient["count"] = false_patient.groupby("id").cumcount()


In [56]:
false_patient["count_1"] = false_patient["count"] - 24
# false_patient["count"] = false_patient.groupby("id")["count"].max()


In [57]:
max_num =  false_patient.groupby("id")["count_1"].max()


In [58]:
max_num = max_num.reset_index()
new_false_patient = pd.merge(false_patient,max_num,how="left",on="id")


In [59]:
new_false_patient.drop(["count_1_x"],axis=1,inplace=True)


In [60]:
import random
def agg(x):
    up = random.randint(0,x["count_1_y"].values[1])
    low = up+24
    
    return x.iloc[up:low,]

df3 = new_false_patient.groupby("id").apply(lambda x: agg(x))


In [61]:
df3 = df3.reset_index(drop=True)
df3.drop(["count","count_1_y"],axis=1,inplace=True)


In [62]:
df3.shape


(172776, 62)

In [63]:
df_12.head()

,ind,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,...,lds,af,AF,time,num,cond,cum_sum,new_count,t2,diff
0,26334,133,63.7,wht,Other,14,5,0,1,8.0,...,0.864124,0.0,0.0,6427.0,1,1,17,1,NaN,15.0
0,26335,133,63.7,wht,Other,14,5,0,1,8.0,...,0.781297,0.0,0.0,6442.0,1,1,18,1,6427.0,15.0
1,26336,133,63.7,wht,Other,14,5,0,1,8.0,...,0.702950,0.0,0.0,6457.0,1,1,19,1,6442.0,15.0
2,26337,133,63.7,wht,Other,14,5,0,1,8.0,...,0.920636,0.0,0.0,6472.0,1,1,20,1,6457.0,15.0
3,26338,133,63.7,wht,Other,14,5,0,1,8.0,...,1.026540,0.0,0.0,6487.0,1,1,21,1,6472.0,15.0


In [64]:
df_12.drop(["num","cond","ind","time","t2","diff","new_count","cum_sum"],axis=1,inplace=True)


In [65]:
total = pd.concat([df3,df_12],axis=0)



In [67]:
total.shape

(178224, 62)

In [68]:
total.head()

,id,age,race,svc,LOS,ICU_Pt_Days,Mort,n_evts,eid,y,...,edrk,s2.edrk,s8.edrk,s24.edrk,srr,dfa,cosen,lds,af,AF
0,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,13.943479,-0.000968,-0.000005,-0.000073,0.025708,0.678520,-2.504942,2.097149,0.0,0.0
1,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,13.583451,-0.001036,-0.000018,-0.000075,0.025586,0.907685,-2.292231,1.772578,0.0,0.0
2,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,14.015315,-0.000860,-0.000036,-0.000079,0.025972,0.827378,-2.445373,1.584857,0.0,0.0
3,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,14.006100,-0.000724,-0.000054,-0.000084,0.025324,0.804359,-2.451298,2.007642,0.0,0.0
4,1,39.35,wht,Cardiothoracic Surgery,4,1,0,0,NaN,False,...,13.745291,-0.000746,-0.000074,-0.000090,0.021729,1.073104,-2.391745,1.697649,0.0,0.0


In [69]:
ecg_col = ["id","y",'hr', 's2.hr', 's8.hr', 's24.hr', 'n.edrk',
       'edrk', 's2.edrk', 's8.edrk', 's24.edrk', 'srr', 'dfa', 'cosen', 'lds',
       'af', 'AF']

In [70]:
new_df = total[ecg_col]


In [71]:
new_df.head()

,id,y,hr,s2.hr,s8.hr,s24.hr,n.edrk,edrk,s2.edrk,s8.edrk,s24.edrk,srr,dfa,cosen,lds,af,AF
0,1,False,80.273637,-0.000337,0.000065,-0.000032,1.0,13.943479,-0.000968,-0.000005,-0.000073,0.025708,0.678520,-2.504942,2.097149,0.0,0.0
1,1,False,79.546667,-0.000469,0.000070,-0.000038,1.0,13.583451,-0.001036,-0.000018,-0.000075,0.025586,0.907685,-2.292231,1.772578,0.0,0.0
2,1,False,79.796440,-0.000499,0.000071,-0.000044,1.0,14.015315,-0.000860,-0.000036,-0.000079,0.025972,0.827378,-2.445373,1.584857,0.0,0.0
3,1,False,77.617353,-0.000756,0.000053,-0.000051,1.0,14.006100,-0.000724,-0.000054,-0.000084,0.025324,0.804359,-2.451298,2.007642,0.0,0.0
4,1,False,78.552222,-0.000825,0.000036,-0.000056,1.0,13.745291,-0.000746,-0.000074,-0.000090,0.021729,1.073104,-2.391745,1.697649,0.0,0.0


In [72]:
new_df["y"]=new_df.id.apply(lambda x: True if x in true_id else False)

In [73]:
train = false_id[0:4788] + true_id[0:150]
test = false_id[4788:] + true_id[150:]
all_data = new_df

In [74]:
X_train = all_data[all_data.id.isin(train)]
x_test = all_data[all_data.id.isin(test)]

In [77]:
# X_train

In [78]:
train_y = X_train[["id","y"]].drop_duplicates()
test_y = x_test[["id","y"]].drop_duplicates()

In [79]:
X_train = X_train.drop(["id","y"],axis=1)
x_test = x_test.drop(["id","y"],axis=1)

In [80]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train= np.array(X_train)
scaler.fit(X_train)
X_train =scaler.transform(X_train)
x_test = np.array(x_test)
x_test = scaler.transform(x_test)

In [82]:
X_train

array([[ 0.13127137, -0.27481118,  0.21914436, ...,  1.11843756,
        -0.31238377, -0.30700945],
       [ 0.08403435, -0.39149335,  0.22750164, ...,  0.57215303,
        -0.31238377, -0.30700945],
       [ 0.10026412, -0.41805928,  0.22919008, ...,  0.25620059,
        -0.31238377, -0.30700945],
       ...,
       [-2.10434865, -0.50312243, -2.70143909, ...,  0.31214959,
        -0.31238377, -0.30700945],
       [-2.24405111, -0.20997143, -2.57985376, ...,  0.31214959,
        -0.31238377, -0.30700945],
       [-2.35927856, -0.02923515, -2.44743883, ...,  0.31214959,
        -0.31238377, -0.30700945]])

In [175]:
X_train = X_train.reshape(4938,24,15)
x_test = x_test.reshape(2488,24,15)


In [83]:
X_train =X_train.reshape(4938,24,15,1) 
x_test = x_test.reshape(2488,24,15,1)

In [84]:
test_id =test_y.id
train_y =train_y.drop("id",axis=1) 
test_y = test_y.drop("id",axis=1)

In [89]:

model= Sequential()
model.add(Conv2D(20, kernel_size=(1,15), input_shape=X_train.shape[1:4]))
model.add(BatchNormalization())
# model.add(MaxPooling2D(3))
# model.add(Conv2D(10, ))
# model.add(BatchNormalization())
# model.add(LeakyReLU(alpha=0.01))
# model.add(Dropout(0.2))
# model.add(Conv2D(128, 2))
# model.add(BatchNormalization())
# model.add(LeakyReLU(alpha=0.01))
# model.add(Dropout(0.2))
# model.add(Conv2D(256, 2))
# model.add(BatchNormalization())
# model.add(LeakyReLU(alpha=0.01))
# model.add(MaxPooling2D(2))
# model.add(Dropout(0.2))
model.add(Flatten())
# model.add(Dense(128))
# model.add(LeakyReLU(alpha=0.01))
# model.add(Dense(10))
# model.add(LeakyReLU(alpha=0.01))
# model.add(Dropout(0.4))
model.add(Dense(1, activation = 'sigmoid'))

In [91]:
nb_epoch = 50
batch_size = 40
model.compile(optimizer='adam', 
                    loss='binary_crossentropy', 
                    metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)
model.fit(X_train, train_y,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(x_test, test_y),
                    verbose=1)

Train on 4938 samples, validate on 2488 samples
Epoch 1/50
4938/4938 [==============================] - 1s 182us/step - loss: 0.1181 - acc: 0.9702 - val_loss: 0.1503 - val_acc: 0.9686
Epoch 2/50
4938/4938 [==============================] - 1s 135us/step - loss: 0.1173 - acc: 0.9688 - val_loss: 0.1519 - val_acc: 0.9691
Epoch 3/50
4938/4938 [==============================] - 1s 146us/step - loss: 0.1163 - acc: 0.9700 - val_loss: 0.1425 - val_acc: 0.9678
Epoch 4/50
4938/4938 [==============================] - 1s 132us/step - loss: 0.1170 - acc: 0.9692 - val_loss: 0.1462 - val_acc: 0.9670
Epoch 5/50
4938/4938 [==============================] - 1s 134us/step - loss: 0.1152 - acc: 0.9700 - val_loss: 0.1447 - val_acc: 0.9662
Epoch 6/50
4938/4938 [==============================] - 1s 129us/step - loss: 0.1140 - acc: 0.9698 - val_loss: 0.1463 - val_acc: 0.9678
Epoch 7/50
4938/4938 [==============================] - 1s 137us/step - loss: 0.1149 - acc: 0.9692 - val_loss: 0.1453 - val_acc: 0.9686


In [92]:
pred = model.predict_proba(x_test)
ind = pred[:,0].argsort()[::-1][:round(len(pred)*0.03)]
test_id = np.array(test_id)

l = list(set(test_id[ind]))
#true patients with event
a = [i for i in l if i in id_true]
len(a)

7

In [180]:
pred

array([[2.2311404e-08],
       [8.4619743e-14],
       [1.0665102e-23],
       ...,
       [1.0858694e-10],
       [6.1605547e-11],
       [2.1081164e-03]], dtype=float32)

In [154]:
pred = model.predict(x_test)
sum(y_test[(pred>0.5).reshape(-1,).tolist()])

17

In [185]:
pre = [False if x <0.5 else True for x in pred ]
from sklearn.metrics import f1_score
f1_score(np.array(test_y), pre)

0.043478260869565216

In [93]:
from sklearn.metrics import roc_auc_score
roc_auc_score(np.array(test_y), pred)

0.617720728048393

In [159]:
#misclassified non_event patients
len(list(set([i for i in test_id[ind] if i  in false_id])))

75

In [160]:
#total unique patients 
len(list(set(test_id)))

2488

In [94]:
X_train = all_data[all_data.id.isin(train)]
x_test = all_data[all_data.id.isin(test)]
train_y = X_train[["id","y"]].drop_duplicates()
test_y = x_test[["id","y"]].drop_duplicates()
X_train = X_train.drop(["id","y"],axis=1)
x_test = x_test.drop(["id","y"],axis=1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train= np.array(X_train)
scaler.fit(X_train)
X_train =scaler.transform(X_train)
x_test = np.array(x_test)
x_test = scaler.transform(x_test)
X_train =X_train.reshape(4938,24,15) 
x_test = x_test.reshape(2488,24,15)
shuffle_idx = np.random.permutation(X_train.shape[1])
X_train = X_train[:,shuffle_idx,:]

X_train =X_train.reshape(4938,24,15,1) 
x_test = x_test.reshape(2488,24,15,1)


In [95]:
test_id =test_y.id
train_y =train_y.drop("id",axis=1) 
test_y = test_y.drop("id",axis=1)

In [96]:
model= Sequential()
model.add(Conv2D(20, kernel_size=(1,15), input_shape=X_train.shape[1:4]))
model.add(BatchNormalization())
# model.add(MaxPooling2D(2))
# model.add(Conv2D(64, 2))
# model.add(BatchNormalization())
# model.add(LeakyReLU(alpha=0.01))
# model.add(Dropout(0.2))
# model.add(Conv2D(128, 2))
# model.add(BatchNormalization())
# model.add(LeakyReLU(alpha=0.01))
# model.add(Dropout(0.2))
# model.add(Conv2D(256, 2))
# model.add(BatchNormalization())
# model.add(LeakyReLU(alpha=0.01))
# model.add(MaxPooling2D(2))
# model.add(Dropout(0.2))
model.add(Flatten())
# model.add(Dense(128))
# model.add(LeakyReLU(alpha=0.01))
# model.add(Dense(10))
# model.add(LeakyReLU(alpha=0.01))
# model.add(Dropout(0.4))
model.add(Dense(1, activation = 'sigmoid'))

In [97]:
nb_epoch = 50
batch_size = 40
model.compile(optimizer='adam', 
                    loss='binary_crossentropy', 
                    metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)
model.fit(X_train, train_y,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(x_test, test_y),
                    verbose=1)

Train on 4938 samples, validate on 2488 samples
Epoch 1/50
4938/4938 [==============================] - 1s 206us/step - loss: 0.4950 - acc: 0.7880 - val_loss: 0.2187 - val_acc: 0.9598
Epoch 2/50
4938/4938 [==============================] - 1s 141us/step - loss: 0.1449 - acc: 0.9692 - val_loss: 0.1443 - val_acc: 0.9678
Epoch 3/50
4938/4938 [==============================] - 1s 143us/step - loss: 0.1320 - acc: 0.9686 - val_loss: 0.1443 - val_acc: 0.9666
Epoch 4/50
4938/4938 [==============================] - 1s 135us/step - loss: 0.1333 - acc: 0.9688 - val_loss: 0.1380 - val_acc: 0.9658
Epoch 5/50
4938/4938 [==============================] - 1s 131us/step - loss: 0.1325 - acc: 0.9680 - val_loss: 0.1407 - val_acc: 0.9674
Epoch 6/50
4938/4938 [==============================] - 1s 137us/step - loss: 0.1296 - acc: 0.9690 - val_loss: 0.1422 - val_acc: 0.9674
Epoch 7/50
4938/4938 [==============================] - 1s 127us/step - loss: 0.1288 - acc: 0.9688 - val_loss: 0.1421 - val_acc: 0.9638


In [98]:
pred = model.predict_proba(x_test)
ind = pred[:,0].argsort()[::-1][:round(len(pred)*0.03)]
test_id = np.array(test_id)
l = list(set(test_id[ind]))
#true patients with event
a = [i for i in l if i in id_true]
len(a)

6

In [99]:
roc_auc_score(np.array(test_y), pred)

0.6454076823218259

In [169]:
sum(y_test[(pred>0.5).reshape(-1,).tolist()])

22

In [170]:
sum(pred>0.5)

array([654])

In [171]:
pre = [False if x <0.5 else True for x in pred ]
from sklearn.metrics import f1_score
f1_score(np.array(y_test), pre)

0.060191518467852256

In [172]:
from sklearn.metrics import precision_recall_curve
precision_recall_curve(np.array(y_test), pred[:,0])

(array([0.03101087, 0.03062047, 0.03063281, ..., 0.        , 0.        ,
        1.        ]),
 array([1.        , 0.98701299, 0.98701299, ..., 0.        , 0.        ,
        0.        ]),
 array([2.91405441e-12, 9.20115893e-12, 1.06033645e-11, ...,
        9.76592243e-01, 9.84086215e-01, 9.93232131e-01], dtype=float32))